In [1]:
import numpy as np
import keras
import pickle
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
from label_data import get_labeled_data

In [3]:
x, y = get_labeled_data('A1', 'A1', 'train_')

In [4]:
print(x.shape)
print(y.shape)

(55915, 30, 20)
(55915,)


In [5]:
ylist = y.tolist()
print(ylist.count(0))
print(ylist.count(1))
print(ylist.count(2))

2573
50697
2645


In [13]:
li = [i for i in range(len(ylist)) if y[i] == 2]

In [27]:
print(x[li[9]])

[[4771750.         4773250.         4771750.         4771750.
  4771750.         4771750.         4771750.         4768250.
  4767250.         4767250.         4768250.         4768220.58823529
  4768250.         4766750.         4766750.         4768035.71428571
  4768250.         4766750.         4766750.         4766750.        ]
 [4766750.         4768250.         4768250.         4768250.
  4766750.         4768250.         4767750.         4768250.
  4767750.         4767500.         4768250.         4766750.
  4766750.         4766750.         4766750.         4766750.
  4768250.         4768250.         4768250.         4768250.        ]
 [4768250.         4766750.         4767250.         4767250.
  4768235.14851485 4766750.         4766750.         4766750.
  4766750.         4766750.         4766750.         4766750.
  4766750.         4768250.         4768235.29411765 4768235.29411765
  4768235.29411765 4768250.         4768250.         4768250.        ]
 [4768250.         